In [1]:
import train as proteo_train
import os
import torch
import torch.nn.functional as F
import torch_geometric
from proteo.datasets.ftd import FTDDataset, reverse_log_transform
import torch.nn.functional as F
import pytorch_lightning as pl
from scipy.stats import zscore
from proteo.checkpoint_analysis import full_load_and_run_and_convert, process_checkpoints

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
csf_all_mean = 2.124088581365514
csf_all_std = 0.8733420033790319

full_load_and_run_and_convert("/home/lcornelis/code/proteo/outputs/ray_results/TorchTrainer_2024-10-17_16-51-50/model=gat-v4,seed=1812_148_act=leaky_relu,adj_thresh=0.7000,batch_size=32,dropout=0.1000,l1_lambda=0.0000,lr=0.0044,lr_scheduler=L_2024-10-17_16-51-51/checkpoint_000094",device, csf_all_mean, csf_all_std)

accumulate_grad_batches=1 trainer_accelerator='gpu' act='tanh' log_every_n_steps=10 use_master_nodes=False lr=0.002255069173338685 wandb_offline=False gat_v4_fc_dropout=[0.1, 0.2, 0.5] precision='32-true' gat={'num_layers': 2, 'hidden_channels': 256, 'heads': 4, 'v2': True} sync_batchnorm=False use_progress_bar=True num_nodes_choices=[7258] gat-v4={'hidden_channels': [64, 128], 'heads': [2, 2], 'use_layer_norm': True, 'which_layer': ['layer1', 'layer2', 'layer3', 'sex', 'mutation'], 'fc_dim': [256, 512, 512, 128], 'fc_dropout': 0.2, 'fc_act': 'tanh', 'weight_initializer': 'orthogonal', 'num_layers': None} output_dir='/home/lcornelis/code/proteo/outputs' lr_scheduler_choices=['LambdaLR', 'ReduceLROnPlateau', 'ExponentialLR', 'StepLR', 'CosineAnnealingLR'] mutation_choices=[['GRN', 'MAPT', 'C9orf72', 'CTL']] grace_period=30 weight_decay=0 y_val='nfl' gat_v4_fc_act=['relu', 'tanh', 'sigmoid', 'leaky_relu', 'elu'] gat_v4_fc_dim=[[64, 128, 128, 32], [128, 256, 256, 64], [256, 512, 512, 128]

[tensor([[  4.5929],
         [  9.8840],
         [  6.8546],
         [  4.8874],
         [  4.6624],
         [ 14.1393],
         [  5.0007],
         [  5.0582],
         [  4.6833],
         [ 12.7460],
         [  5.3799],
         [  3.0770],
         [  4.5473],
         [  6.5572],
         [  5.1148],
         [  4.4350],
         [  6.8056],
         [  6.4788],
         [  4.8999],
         [  4.4825],
         [  4.3170],
         [  4.4458],
         [  5.0001],
         [  5.2328],
         [  4.2541],
         [  4.5044],
         [  4.3769],
         [  4.2490],
         [ 23.8662],
         [  3.7375],
         [  8.5496],
         [  4.5749],
         [ 26.2937],
         [  4.6610],
         [  3.8351],
         [  4.9153],
         [  4.1966],
         [  4.7331],
         [  5.3969],
         [  4.5096],
         [  5.0424],
         [  4.7726],
         [  6.4580],
         [  5.3791],
         [  8.0433],
         [  5.2280],
         [  6.9078],
         [  3

In [8]:
# Sex_specific_adj no sex
checkpoint1 = "/home/lcornelis/code/proteo/outputs/ray_results/TorchTrainer_2024-10-15_15-51-21/model=gat-v4,seed=15680_180_act=tanh,adj_thresh=0.3500,batch_size=32,dropout=0.1000,l1_lambda=0.0007,lr=0.0005,lr_scheduler=Lambda_2024-10-15_15-51-23/checkpoint_000001"
checkpoint2 = "/home/lcornelis/code/proteo/outputs/ray_results/TorchTrainer_2024-10-15_15-51-21/model=gat-v4,seed=60527_226_act=tanh,adj_thresh=0.9000,batch_size=32,dropout=0,l1_lambda=0.0025,lr=0.0006,lr_scheduler=LambdaLR,mo_2024-10-15_17-14-43/checkpoint_000006"
full_load_and_run_and_convert(checkpoint1 ,device, csf_all_mean, csf_all_std)

adj_thresh_choices=[0.1, 0.2, 0.35, 0.5, 0.7, 0.9] dropout=0.1 gat-v4={'hidden_channels': [8, 16], 'heads': [4, 4], 'use_layer_norm': True, 'which_layer': ['layer1', 'layer2', 'layer3', 'sex', 'mutation'], 'fc_dim': [128, 256, 256, 64], 'fc_dropout': 0.1, 'fc_act': 'relu', 'weight_initializer': 'orthogonal', 'num_layers': None} gat_hidden_channels=[8, 32, 128, 256] output_dir='/home/lcornelis/code/proteo/outputs' lr_min=1e-06 gcn_num_layers=[2, 3, 4] lr_scheduler='LambdaLR' nodes_count=1 raw_file_name='ALLFTD_dataset_for_nina_louisa_071124_age_adjusted.csv' gpu_per_worker=1 gcn={'num_layers': 3, 'hidden_channels': 32} num_workers=16 trainer_accelerator='gpu' use_progress_bar=True gat_num_layers=[2, 4, 6, 12] modality_choices=['csf'] batch_size=32 seed=15680 lr_scheduler_choices=['LambdaLR', 'ReduceLROnPlateau', 'ExponentialLR', 'StepLR', 'CosineAnnealingLR'] gat_heads=[1, 2, 4, 8] reduction_factor=6 num_nodes=7258 lr=0.0005472969709054848 master_nodes=['sex', 'mutation', 'age'] epochs=

[tensor([[ 3.0691],
         [ 6.8723],
         [ 9.8589],
         [ 7.3550],
         [ 6.8568],
         [ 6.8783],
         [ 4.7050],
         [ 3.0802],
         [ 7.3576],
         [ 7.3862],
         [ 9.8421],
         [ 7.3662],
         [ 4.7106],
         [ 7.3690],
         [13.7894],
         [ 7.3655],
         [ 7.3687],
         [ 3.0739],
         [ 4.7164],
         [ 4.7087],
         [ 4.7094],
         [ 4.7072],
         [ 3.0756],
         [ 9.8390],
         [ 6.8600],
         [ 7.3520],
         [ 3.0737],
         [ 3.0739],
         [ 6.4692],
         [ 3.0708],
         [ 7.3678],
         [ 7.3572],
         [ 6.8779],
         [ 7.3677],
         [ 4.7093],
         [ 3.0758],
         [ 7.3440],
         [ 3.0717],
         [ 4.7107],
         [ 3.0709],
         [ 6.4661],
         [ 4.7161],
         [ 4.7132],
         [ 7.3653],
         [13.8137],
         [ 9.8529],
         [13.7935],
         [ 6.8705],
         [ 6.8549],
         [13.8172],


In [14]:
#Sanity check
def compute_manual_mse(val_preds, val_targets):
    """
    Manually computes the Mean Squared Error (MSE) for the given predictions and targets.

    Parameters:
    val_preds (list of list of torch.Tensor): The predicted values.
    val_targets (list of list of torch.Tensor): The true target values.

    Returns:
    float: The computed Mean Squared Error.
    """
   # Compute the squared differences
    squared_diffs = (val_preds - val_targets) ** 2

    # Compute the mean of the squared differences
    mse = squared_diffs.mean().item()

    return mse

print(compute_manual_mse(val_preds, val_targets))

716.7650146484375


In [13]:
import os
import torch
import torch.nn.functional as F
import train as proteo_train

# Define a function to load the checkpoint and calculate MSE
def load_checkpoint_and_calculate_mse(relative_checkpoint_path, levels_up=5):
    # Get the current script directory
    current_directory = os.getcwd()
    
    # Navigate up the specified number of levels
    for _ in range(levels_up):
        current_directory = os.path.dirname(current_directory)
    
    # Construct the full path to the checkpoint
    checkpoint_path = os.path.join(current_directory, relative_checkpoint_path)
    print(f"Loading checkpoint from: {checkpoint_path}")

    # Check if the file exists to avoid errors
    if not os.path.isfile(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint file not found: {checkpoint_path}")

    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)
    print("Checkpoint keys:", checkpoint.keys())
    print("checkpoint state_dict keys:", checkpoint['state_dict'].keys())

    module = proteo_train.Proteo.load_from_checkpoint(checkpoint_path)

    # Access the attributes
    # best_val_pred = module.best_val_pred
    # print("best_val_pred:", best_val_pred)
    # # print("min_val loss:", module.val_loss)
    # best_val_target = module.best_val_target
    # best_train_pred = module.best_train_pred
    # best_train_target = module.best_train_target

    # # Calculate MSE for validation and training
    # mse_val = F.mse_loss(best_val_pred, best_val_target).item()
    # mse_train = F.mse_loss(best_train_pred, best_train_target).item()

    return module, checkpoint

# Example usage
relative_checkpoint_path = '/scratch/lcornelis/outputs/ray_results/TorchTrainer_2024-07-31_16-47-02/model=gat-v4,seed=19543_0_act=relu,adj_thresh=0.1000,batch_size=8,dropout=0,l1_lambda=0.0000,lr=0.1000,lr_scheduler=LambdaLR,modal_2024-07-31_16-47-02/checkpoint_000001/checkpoint.cpkt'
module, checkpoint = load_checkpoint_and_calculate_mse(relative_checkpoint_path)
# print(f"MSE Loss for validation set: {mse_val}")
# print(f"MSE Loss for training set: {mse_train}")


Loading checkpoint from: /scratch/lcornelis/outputs/ray_results/TorchTrainer_2024-07-31_15-31-35/model=gat-v4,seed=19543_0_act=relu,adj_thresh=0.1000,batch_size=8,dropout=0,l1_lambda=0.0000,lr=0.1000,lr_scheduler=LambdaLR,modal_2024-07-31_15-31-36/checkpoint_000001/checkpoint.ckpt
Checkpoint keys: dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])
checkpoint state_dict keys: odict_keys(['model.convs.0.att_src', 'model.convs.0.att_dst', 'model.convs.0.bias', 'model.convs.0.lin.weight', 'model.convs.1.att_src', 'model.convs.1.att_dst', 'model.convs.1.bias', 'model.convs.1.lin.weight', 'model.pools.0.weight', 'model.pools.0.bias', 'model.pools.1.weight', 'model.pools.1.bias', 'model.layer_norm.weight', 'model.layer_norm.bias', 'model.encoder.0.0.weight', 'model.encoder.0.0.bias', 'model.encoder.1.0.weight', 'model.encoder.1.0.bias', 'model.encoder.2.0.weight', 'model.